# GTI771 - Apprentissage machine avancé
## Département de génie logiciel et des technologies de l’information



## Laboratoire 5 - Régression linéaire
#### <font color=black> Version 2 - Été 2024 </font>

##### <font color=grey> Version 1 - Prof. Alessandro L. Koerich.
##### Version 2 - Chargé de lab. Arthur Josi

| NOMS                  | CODE PERMANENT  |  PARTICIPATION     |
|-----------------------|-----------------|--------------------|
| Étudiant1             | Code1           |      0%            |
| Étudiant2             | Code2           |      0%            |
| Étudiant3             | Code3           |      0%            |

## Introduction
Dans ce laboratoire, vous êtes amenés à utiliser des algorithmes de régression aﬁn de résoudre le problème de prédiction de l'âge de personnes à partir de photos du visage.

Le problème de régression qui vous est présenté est le problème [Facial Aging Estimation (FAE)](https://yanweifu.github.io/FG_NET_data/index.html), dont le but est de prédire l'âge des personnes à partir du visage. En vous basant sur les concepts vus en classe et l'expérience acquise dans le laboratoires précedents, vous êtes invité à l’extraction de primitives puis la regression de l'âge sur l’ensemble de données fourni avec cet énoncé.

##### Description de l'ensemble de données FG-NET:
* 1002 images faciales de 82 sujets multiraciaux âgés de 0 à 69 ans;
* Déséquilibré: 50% des sujets ont entre 0 et 13 ans;
* Images couleur et niveaux de gris avec largeur entre 300 et 359 pixels, hauteur entre 639 et 772 pixels, et résolution entre 200 dpi et 1200 dpi;
* Grande variation d'éclairage, de pose, d'expression faciale, de flou et d'occlusions (par exemple, moustache, barbe, lunettes, etc.).

Voici, en exemple, des images de visages se retrouvant dans l’ensemble de données FG-NET:

![Exemples de FG-NET](https://www.mdpi.com/sensors/sensors-16-00994/article_deploy/html/images/sensors-16-00994-g001.png)

L’évaluation de ce laboratoire sera basée sur:
- la qualité des algorithmes proposés et utilisés; (10%)
- utilisation du protocole et mesures de performance appropriées; (10%)
- les réponses aux questions dans ce notebook (Les cellules dans votre PDF ou votre notebook doivent être processées dans votre rendu); (70%)
- l'organisation de votre code source (n'oubliez pas de mettre des commentaires dans le code source!) (10%)

# Modules et bibliotèques python

### Import de bibliotèques

###  <font color=blue> À faire: </font>
1. Ajouter les bibliothèques que vous avez utilisées pour compléter ce notebook dans une cellule avec une petite description.

In [29]:
import numpy as np  # package for scientific computing with Python.
import matplotlib.pyplot as plt # 2D plotting library
import os
import re
from PIL import Image
import numpy as np
import re 

### Définition des fonctions

In [30]:
def fa():
    return 1

# Partie 1 - Lecture des images et préparation des données (40%)

Point de départ: 1002 images jpeg de l'ensemble FG-NET

## 1a: Lecture, nettoyage, prétraitement, normalisation et annotation

Différemment de l'ensemble FER, l’ensemble FG-NET consiste en un répertoire avec 1,002 images JPEG, où les étiquettes sont les noms des fichiers (âge et l'id du sujet).

Vous devez lire ces images et les représenter sous la forme d’une matrice $X\_data$ aussi que transformer les noms des fichiers dans un vecteur $Y\_data$ avec les âges et un troisième vecteur $Z\_data$ avec les id des sujets.

Vous devez également, comme vous avez déjà fait pour l'ensemble de données FER, vous assurer qu’il n'y a pas:
- données aberrantes;
- valeurs manquantes;
- valeurs inapplicables ou aberrantes;
- etc.  
PS: Pour rebalancer, ne faite pas un simple upsampling, augmentez vos données.

Finalement, vous devez également appliquer de prétraitement pour réduire la variabilité, réduire des bruits, etc. En particulier, pour les images de visage, quelques prétraitements peuvent se montrer utiles, comme:
- Localisation/recadrage du visage?
- Localisation des yeux?
- Lissage du visage?
- Normalization du contraste?
- Etc.

###  <font color=blue> À faire: </font>

1. Lire les images jpeg et les noms des fichiers et les représenter sous la forme de matrices de pixels et vecteurs de étiquettes ($X\_data$, $Y\_data$, $Z\_data$ comme décrit précédemment).

In [31]:
string = "001A002a.JPG"

id, age = getId_Age_from_fileName(string)

print("id = ", id)
print("age = ", age)

id =  1
age =  2


In [32]:
import os
import re
from PIL import Image
import numpy as np
import re 


def getId_Age_from_fileName(fileName):
    s1 = fileName.split(".")
    
    s2 = s1[0].split("A")
    s2[1] = re.sub('[^0-9]','', s2[1])
    
    id = int(s2[0])
    age = int(s2[1])

    return id, age


foldername = "fgnet"
dir_list = os.listdir(foldername)
print(len(dir_list))

# Initialize the data arrays
X_data = np.zeros((len(dir_list), 256, 256, 3), dtype=np.uint8)
Y_data = np.zeros((len(dir_list), 1), dtype=int)
Z_data = np.zeros((len(dir_list), 1), dtype=int)


dir_list.sort()

inti = 0
count = 0
for file in dir_list:
    filename = file

    id, age = getId_Age_from_fileName(filename)
    
    if id is not None and age is not None:
        Z_data[inti] = id
        Y_data[inti] = age
        
        # Open the image and resize to 256x256
        img_path = os.path.join(foldername, filename)
        with Image.open(img_path) as img:
            img = img.resize((256, 256))
            img_array = np.array(img)

            if img_array.ndim == 2:  # Grayscale image
                img_array = np.stack((img_array,) * 3, axis=-1)
            
            if img_array.shape == (256, 256, 3):
                X_data[inti] = img_array
            else:
                count += 1
                #print(f"Image shape mismatch for file: {filename}")
        
        inti +=1


print("Total images with shape mismatch: ", count)
print(X_data.shape)
print(Y_data.shape)
print(Z_data.shape)

for i in range(0, 10):
    print("ID = ", Z_data[i], " Age = ", Y_data[i])


1002
Total images with shape mismatch:  0
(1002, 256, 256, 3)
(1002, 1)
(1002, 1)
ID =  [1]  Age =  [2]
ID =  [1]  Age =  [5]
ID =  [1]  Age =  [8]
ID =  [1]  Age =  [10]
ID =  [1]  Age =  [14]
ID =  [1]  Age =  [16]
ID =  [1]  Age =  [18]
ID =  [1]  Age =  [19]
ID =  [1]  Age =  [22]
ID =  [1]  Age =  [28]



2. Concevez et codifiez un algorithme pour vérifier l'intégrité des données, faire des corrections si nécessaires.


In [38]:
# Not sure how to do this

for images in X_data:
    #do something to check if the images are okay
    if images.shape != (256, 256, 3):
        print("Image shape is not correct")
    if images.dtype != np.uint8:
        print("Image type is not uint8")
    if images.max() > 255 or images.min() < 0:
        print("Image pixel values are out of range")
    mean = np.mean(images)
    if mean < 0 or mean > 255:
        print("Image mean is out of range")
    

for age in Y_data:
    #do something to check if the ages are okay
    if age<0 or age > 100:
        print("Age is out of range")

for id in Z_data:
    #do something to check if the ids are okay
    if id<0 or id > 1000:
        print("ID is out of range")


3. Appliquez au moins un prétraitement sur les images de visages. Vous pouvez choisir différents algorithmes de prétraitement d’images dans [scikit-image](https://scikit-image.org/docs/stable/api/api.html) ou dans la librairie de votre choix. Vous pouvez aussi trouver d’autres types de prétraitement qui sont plus généraux dans [scikit-learn](https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing). Vous pouvez utiliser les mêmes prétraitements que ceux utilisés avec FER.


4. Choisir et appliquer une résolution $n\times m$ qui vous semble pertinente pour normaliser les images, car celles-ci n'ont pas toutes la même résolution.

5. Transformez toutes les images afin que celles-ci soient toutes définies sur les 3 canaux (RGB).


6. Générez un fichier *fg-net-nxm.csv* avec les données nettoyées et normalisées, où $n$ et $m$ représentent la résolution finale des images.
   - Format du fichier: subject,age,pixels
      * sujet: integer
      * âge: integer
      * pixels: integer [0, 255]

7. Décrivez brièvement les étapes de votre algorithme/code.

8. Créer une grille de dimension 7$\times$2 avec des images de visage prises aléatoirement de l'ensemble FG-NET original et après vos traitements. Afficher également l'âge et le id du sujet au dessus de chaque image.

# Partie 2 - Extraction de primitives (10%)

###  <font color=blue> À faire: </font>
1. Choisir et extraire un jeu de primitives pour représenter les images de FG-NET. Vous êtes fortement conseillé de choisir
le jeu de primitives les plus performants dans les TPs précédents (primitives artisanaux, deep, deep réduit, etc.)

2. Sauvegardez vos vecteurs de primitives sous la forme d'un fichier 'csv' (p. ex. *fgnet-deepVGG19.csv*). N'oubliez pas d'utiliser toujours la même structure du fichier *fg-net-nxm.csv* de 1a). Vous devez nommer vos fichiers de primitives en référence au jeu de primitives utilisé, p. ex., *fg-net-12x12-deepVGG19.csv* pour des primitives produites avec une CNN VGG19.


3. Décrivez très brièvement le choix du jeu de primitives.

# Partie 3: Entraînement de modèles de régression (20%)

Vous êtes maintenant prêtes à entraîner un modèle d'apprentissage automatique avec les vecteurs de primitives extraits dans la Partie 2.

###  <font color=blue> À faire: </font>
1. Choisir deux (2) algorithmes de régression disponibles dans Scikit-learn:
    * Régression lineaire
    * Régression Ridge
    * Régression Lasso et Elastic-Net
    * Descente du gradiente stochastique (SGD)<br>

Conseil: Étudiez les algorithmes choisis pour bien comprendre les différents hyperparamètres qui peuvent affecter l'entraînement, la généralisation et la complexité du modèle de régression.

Algorithmes choisis: \<ici\>

2. Entraîner et optimiser les paramètres des modèles de régression. Utiliser le protocole <font color=blue> "Leave One Subject Out Cross-Validation" </font> (LOSO).

3. Étudiez les metriques d'évaluation des modèles de régression disponibles dans [Scikit-Learn](https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics) puis utilisez MSE, MAE et une troisième metrique de votre choix.


4. Faire une brève analyse des résultats et présenter vos considérations et conclusions sur les algorithmes de régression choisis.

| Algorithme            | Paramètres    |  MSE  |  MAE  | \<votre choix\> |
|-----------------------|---------------|-------|-------|-------|
| Regr lineaire         | XXX.XX        |XXX.XX |XXX.XX |       |
| Regr Ridge            | alpha = 0.1   |123.34 | 10.45 |       |
| Regr Lasso            | XXX.XX        |XXX.XX |XXX.XX |       |
| Regr ElasticNet       | XXX.XX        |XXX.XX |XXX.XX |       |
| ...                   | XXX.XX        |XXX.XX |XXX.XX |       |
| ...                   | XXX.XX        |XXX.XX |XXX.XX |       |

# Fin